In [7]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [8]:
words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [9]:
len(words)

32033

In [10]:
chars = sorted(list(set("".join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [11]:
block_size = 3
X, Y = [], []
for w in words:
    context = [0] * block_size
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        #print(''.join(itos[i] for i in context), '--->', itos[ix])
        context = context[1:] + [ix]
X = torch.tensor(X)
Y = torch.tensor(Y)

In [12]:
block_size = 3
def build_dataset(split):
    X, Y = [], []
    for w in split:
        c = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(c)
            Y.append(ix)
            c = c[1:] + [ix]
    return torch.tensor(X), torch.tensor(Y)

In [13]:
import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xtest, Ytest = build_dataset(words[n2:])

print(Xtr.shape)
print(Xdev.shape)
print(Xtest.shape)

torch.Size([182625, 3])
torch.Size([22655, 3])
torch.Size([22866, 3])


In [14]:
Y

tensor([ 5, 13, 13,  ..., 26, 24,  0])

In [15]:
X.shape

torch.Size([228146, 3])

In [16]:
C = torch.rand((27, 2))

In [17]:
enb = C[X]
enb.shape

torch.Size([228146, 3, 2])

In [18]:
W1 = torch.rand((6, 100))
b1 = torch.rand(100)

In [19]:
enb.view(32, 6)

RuntimeError: shape '[32, 6]' is invalid for input of size 1368876

In [ ]:
h = enb.view(-1, 6) @ W1 + b1
h

In [ ]:
h = torch.tanh(h)
h.shape

In [ ]:
W2 = torch.randn((100, 27))
b2 = torch.randn(27)

In [ ]:

logits = h @ W2 + b2

In [ ]:
logits.shape

In [ ]:
counts = logits.exp()
probs = counts / counts.sum(1, keepdims=True)
probs.shape

In [ ]:
-probs[torch.arange(32), Y].log().mean()

In [30]:
g = torch.Generator().manual_seed(214748647)
C = torch.randn((27,15), generator=g)
W1 = torch.rand((45, 200), generator=g)
b1 = torch.randn(200, generator=g)
W2 = torch.randn((200, 27), generator=g)
b2 = torch.randn(27, generator=g)
parameters = [C, W1, b1, W2, b2]

In [31]:
sum(p.nelement() for p in parameters)

15032

In [32]:
for p in parameters:
    p.requires_grad = True

In [34]:
for i in range(50000):
    ix = torch.randint(0, Xtr.shape[0], (128,))
    enb = C[Xtr[ix]]
    h = torch.tanh(enb.view(-1, 45) @ W1 + b1)
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits, Ytr[ix])
    for p in parameters:
        p.grad = None
    loss.backward()
    for p in parameters:
        p.data += -0.01 * p.grad
print(loss.item())

2.015265941619873


In [36]:
enb = C[Xtr]
h = torch.tanh(enb.view(-1, 45) @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Ytr)
loss

tensor(2.1133, grad_fn=<NllLossBackward0>)

In [37]:
enb = C[Xdev]
h = torch.tanh(enb.view(-1, 45) @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Ydev)
loss

tensor(2.1627, grad_fn=<NllLossBackward0>)

In [ ]:
sampleGen = torch.generator().manual_seed(2147483647 + 10)
for _ in range(20):
    
    out = []
    context = [0] * block_size
    while True:
        enb = C[torch.tensor([context])]
        h = torch.tanh(enb.view(1, -1) @ W1 + b1)
        logits = h @ W2 + b2
        probs = F.softmax(logits, dim=1)
        ix = torch.multinomial(probs, num_samples=1, generator=sampleGen).item()
        context = context[1:] + ix
        out.append(ix)
        if ix == 0:
            